## libreries

In [1341]:
import re
import pdfplumber
import spacy
from spacy.training import Example
import random
import os
import json

## Fecha

In [1342]:
def parse_date_str(text: str) -> str | None:
    patterns = [
        r"(\d{1,2})\s*(?:de\s*)?([A-Z]+)\s*(?:de\s*)?(\d{4})",
        r"(\d{1,2})\s+([A-Z]{3,})\s+(\d{4})",
    ]
    month_map = {
        "ENE": "01", "FEB": "02", "MAR": "03", "ABR": "04",
        "MAY": "05", "JUN": "06", "JUL": "07", "AGO": "08",
        "SEP": "09", "OCT": "10", "NOV": "11", "DIC": "12",
        "ENERO": "01", "FEBRERO": "02", "MARZO": "03", "ABRIL": "04",
        "MAYO": "05", "JUNIO": "06", "JULIO": "07", "AGOSTO": "08",
        "SEPTIEMBRE": "09", "OCTUBRE": "10", "NOVIEMBRE": "11", "DICIEMBRE": "12"
    }

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                day = match.group(1).zfill(2)
                month = month_map.get(match.group(2).upper())
                year = match.group(3)

                if month:
                    return f"{day}/{month}/{year}"
            except ValueError:
                continue

    return None

In [1343]:
def looks_like_date(candidate: str) -> bool:
    """
    Verifica si 'candidate' se puede parsear como fecha 
    usando la función parse_date_str.
    Retorna True si es una fecha válida, de lo contrario False.
    """
    return parse_date_str(candidate) is not None

In [1344]:
def normalize_spacy_date(spacy_fecha: str) -> str | None:
    """
    Intenta parsear la fecha que spaCy devolvió y, si es válida,
    retorna la versión normalizada (dd/mm/yyyy).
    De lo contrario, retorna None.
    """
    if not spacy_fecha:
        return None
    
    parsed = parse_date_str(spacy_fecha)
    return parsed  


## Extrae epigrafe

In [1345]:
def extraer_epigrafe(text: str, fecha_normalizada: str) -> str | None:
    if fecha_normalizada:
        pattern = rf"{re.escape(fecha_normalizada)}.*?(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"
    else:
        pattern = r"(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"

    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

## tipo de ley 

In [1346]:
def fix_ocr_issues(text: str) -> str:
    correcciones = {
        "NACiÓN": "NACIÓN",
        "Colr,mbiana": "Colombiana",
        "Perscnal": "Personal",
        "AérEa": "Aérea",
        "No1956": "No 1956",
        "No.1956": "No. 1956",
        "N°1956": "N° 1956",
        "LEYNo": "LEY No",
        "LEYNo.": "LEY No.",
        "4JUN2019": "4 JUN 2019",
        "Dadaen": "Dada en"
    }
    for error, correccion in correcciones.items():
        text = text.replace(error, correccion)
        
    # Filtramos saltos de línea basura
    lines = text.split("\n")
    clean_lines = []
    for line in lines:
        # Si la línea está compuesta solo de guiones, tildes, espacios o está vacía,
        # la consideramos basura y la saltamos.
        # Ajusta la regex a tus necesidades.
        if re.match(r"^[\-\~\s]+$", line.strip()):
            continue
        clean_lines.append(line)

    text = "\n".join(clean_lines)
    text = re.sub(r"(?<=\S)\n(?=\S)", " ", text)
    text = re.sub(r"\n{2,}", "\n\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = text.strip()
    text = re.sub(r"(No|N°)(\d+)", r"\1 \2", text, flags=re.IGNORECASE)
    text = re.sub(r"(\d+)([A-Z]{1,3}\d{4})", r"\1 \2", text)
    return text

def parse_ley_and_type(text: str):
    type_map = {
        "orgánica": "orgánica",
        "organica": "orgánica",
        "ordinaria": "ordinaria",
        "estatutaria": "estatutaria",
        "estaturaria": "estatutaria",
    }

    pattern = r"""
        (?:LEY|LEYES)[\s\n]*
        (?:
            (ORG[ÁA]NICA|ORGANICA|ORDINARIA|ESTATUTARIA|ESTATURARIA)
            [\s\n]+
        )?
        (?:N|No|N°|No\.)?
        [\s\n]*
        (\d+)
    """
    match = re.search(pattern, text, re.IGNORECASE | re.VERBOSE | re.DOTALL)

    if not match:
        return (None, None)

    raw_type = match.group(1)
    ley_number = match.group(2)

    if raw_type:
        tipo_ley = type_map.get(raw_type.lower())
    else:
        tipo_ley = None

    return (tipo_ley, ley_number)

## train epigrafe

In [1347]:
TRAIN_DATA = [
    (
        # Ejemplo 1
        '2353 17ABR2024 LEY  No. "POR  MEDIO  DE  LA  CUAL  SE  RECONOCE  COMO  PATRIMONIO '
        'CULTURAL  INMATERIAL  DE  LA NACION  EL  FESTIVAL  PROVINCIANO  DE  ACORDEONES,  CANCION '
        'INÉDITA  Y  PIQUERIA,  DEL  MUNICIPIO  DE  PIVIJAY  -  MAGDALENA  Y  SE  DICTAN  OTRAS '
        'DISPOSICIONES" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2353"
            (5, 15, "FECHA"),              # "17ABR2024"
            (25, 272, "EPIGRAFE")          # desde la apertura de comillas hasta justo antes de "El Congreso..."
        ]}
    ),
    (
        # Ejemplo 2 (nota: la fecha se compone de "17MAY" y "2024" separados en el texto; en la anotación se unen)
        '2355 2024 LEY  No. 17MAY "POR  MEDIO  DEL  CUAL  SE  RECONOCE  EL  FESTIVAL  DEPARTAMENTAL '
        'DE  BANDAS  DE  CUNDINAMARCA  COMO  MANIFESTACION  DEL  PATRIMONIO  CULTURAL  INMATERIAL '
        'DE  LA  NACION" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2355"
            (5, 9, "FECHA"),              # "2024" puede interpretarse junto a "17MAY" (ver nota abajo)
            (19, 24, "FECHA"),            # "17MAY"
            (25, 196, "EPIGRAFE")         # epígrafe entre comillas
        ]}
    ),
    (
        # Ejemplo 3: La fecha viene separada en tres líneas ("2356", "8", "2024" y luego "MAY")
        '2356 8 2024 LEY  No. MAY "POR  MEDIO  DE  LA  CUAL  SE  ELIMINAN  BENEFICIOS  Y  SUBROGADOS '
        'PENALES  PARA  QUIENES  SEAN  CONDENADOS  O  ESTÉN  CUMPLIENDO  DETENCION  PREVENTIVA '
        'POR  EL  DELITO  DE  FEMINICIDIO" El  Congreso  de  Colombia,',
        {"entities": [
            (0, 4, "NUMERO_LEY"),         # "2356"
            (5, 6, "FECHA"),              # "8" (día)
            (7, 11, "FECHA"),             # "2024" (año)
            (21, 24, "FECHA"),            # "MAY" (mes)
            (25, 211, "EPIGRAFE")
        ]}
    ),
    (
        # Ejemplo 4: Uso de "EL CONGRESO DE LA REPUBLICA DE COLOMBIA" para marcar el fin del epígrafe
        'LEY  No.2361 14JUNZUZ4 POR  MEDIO  DEL  CUAL  SE  OTORGAN  LINEAMIENTOS  PARA  LA '
        'CREACION  DE  LA  POLITICA  PUBLICA  DE  LACTANCIA  MATERNA,  ALIMENTACION  COMPLEMENTARIA,  '
        'Y  LA  PROMOCION  DE  LOS  BANCOS  DE  LECHE  HUMANA  COMO  COMPONENTE  ANATOMICO  '
        'EL  CONGRESO  DE  LA  REPUBLICA  DE  COLOMBIA  DECRETA:',
        {"entities": [
            (8, 12, "NUMERO_LEY"),         # "2361"
            (13, 22, "FECHA"),              # "14JUNZUZ4" (aquí convendría aplicar correcciones manuales o reglas post-procesado)
            (23, 258, "EPIGRAFE")           # Desde "POR MEDIO DE..." hasta antes de "EL CONGRESO..."
        ]}
    ),
    (
        # Ejemplo 5: Problema en la fecha "14JU Te" y epígrafe sin comillas
        '14JU Te LEY  No.  2360 POR  MEDIO  DE  LA  CUAL  SE  MODIFICA  Y  ADICIONA  LA  LEY  1384  DE  2010 '
        'RECONOCIENDO  PARA  LOS  EFECTOS  DE  ESTA  LEY  COMO  SUJETOS  DE  ESPECIAL  PROTECCION  CONSTITUCIONAL  A  LAS  '
        'PERSONAS  CON SOSPECHA  O  QUE  PADECEN  CANCER EL  CONGRESO  DE  LA  REPUBLICA DECRETA:',
        {"entities": [
            (0, 4, "FECHA"),
            (18, 22, "NUMERO_LEY"),         
            (23, 261, "EPIGRAFE")
        ]}
        
        # # Ejemplo 5: Problema en la fecha "14JU Te" y epígrafe sin comillas
        # '8 FEB 2022\n-\nLEY ORGÁNICA No2199\nLEY ORGANICA No2199 FEB 2022 --------~~~~~~~~~--- "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTÍCULO 325 DE LA "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTICULO LA CONSTITUCIÓN POLÍTICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA CONSTITUCION POLITICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA REGIÓN METROPOLITANA BOGOTÁ - CUNDINAMARCA" REGION METROPOLITANA BOGOTA CUNDINAMARCA" EL CONGRESO DE COLOMBIA EL CONGRESO DE COLOMBIA DECRETA: ',
        # {"entities": [
        #     (0, 4, "FECHA"),
        #     (18, 22, "NUMERO_LEY"),         
        #     (23, 261, "EPIGRAFE")
        # ]}
    )
]


In [1348]:
# example_text = '8 FEB 2022 LEY ORGÁNICA No2199 LEY ORGANICA No2199 FEB 2022 --------~~~~~~~~~--- "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTÍCULO 325 DE LA "POR MEDIO DE LA CUAL SE DESARROLLA EL ARTICULO LA CONSTITUCIÓN POLÍTICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA CONSTITUCION POLITICA Y SE EXPIDE EL RÉGIMEN ESPECIAL DE LA REGIÓN METROPOLITANA BOGOTÁ - CUNDINAMARCA" REGION METROPOLITANA BOGOTA CUNDINAMARCA" EL CONGRESO DE COLOMBIA EL CONGRESO DE COLOMBIA DECRETA'
# print("Texto completo:", example_text)
# print("Longitud total:", len(example_text))
# print("Substring(19,230):", example_text[25:272])


In [1349]:
def train_spacy_model(train_data, output_dir="modelo_leyes_epigrafe"):
    # Crea un pipeline spaCy en blanco (español)
    nlp = spacy.blank("es")
    
    # Agrega el componente 'ner'
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    else:
        ner = nlp.get_pipe("ner")

    # Añadir las etiquetas a reconocer
    for text, annotations in train_data:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

    # Inicializar el pipeline
    nlp.initialize()

    # Crear una lista de Example
    examples = []
    for text, annotations in train_data:
        # Crea un doc a partir del texto
        doc = nlp.make_doc(text)
        # Crea un Example que contenga el doc y la anotación
        example = Example.from_dict(doc, annotations)
        examples.append(example)

    # Entrenar
    for i in range(10):  # número de épocas
        random.shuffle(examples)
        losses = {}
        for example in examples:
            nlp.update([example], losses=losses)
        print(f"Época {i}, pérdidas: {losses}")

    # Guardar el modelo entrenado
    nlp.to_disk(output_dir)
    print(f"Modelo guardado en: {output_dir}")

    return nlp


## Define pipeline

In [1350]:
# Entrenar el modelo con TRAIN_DATA
nlp_trained = train_spacy_model(TRAIN_DATA, "modelo_leyes_epigrafe")

Época 0, pérdidas: {'ner': 296.1709858030081}
Época 1, pérdidas: {'ner': 100.17194815090625}
Época 2, pérdidas: {'ner': 48.91475365113858}
Época 3, pérdidas: {'ner': 106.29416485578864}
Época 4, pérdidas: {'ner': 13.568746242703316}
Época 5, pérdidas: {'ner': 81.31574794338525}
Época 6, pérdidas: {'ner': 218.01332047967202}
Época 7, pérdidas: {'ner': 22.61748946640562}
Época 8, pérdidas: {'ner': 68.36770297657034}
Época 9, pérdidas: {'ner': 11.486735263148187}
Modelo guardado en: modelo_leyes_epigrafe


## Cargar el modelo entrenado y crear spacy_extract_metadata


In [1351]:
def load_spacy_model(model_path="modelo_leyes_epigrafe"):
    nlp = spacy.load(model_path)
    return nlp

def spacy_extract_metadata(text: str, nlp) -> dict:
    doc = nlp(text)
    result = {
        "numero_ley": None,
        "fecha": None,
        "epigrafe": None
    }
    for ent in doc.ents:
        if ent.label_ == "NUMERO_LEY":
            result["numero_ley"] = ent.text
        elif ent.label_ == "FECHA":
            result["fecha"] = ent.text
        elif ent.label_ == "EPIGRAFE":
            result["epigrafe"] = ent.text
    return result


In [1352]:
def looks_like_number_ley(value): 
    # Chequeo rápido: si no hay dígitos, no es un número de ley válido
    return bool(re.search(r"\d+", value))

## Fallback

In [1353]:
def extraer_epigrafe_fallback(text: str, final_fecha: str | None) -> str | None:
    """
    Intenta extraer el epígrafe anclado a la fecha (si la hay).
    Si no encuentra nada, hace fallback a la búsqueda sin fecha.
    """
    # 1) Si hay fecha final, probamos la regex anclada a la fecha
    if final_fecha:
        pattern_with_date = rf"{re.escape(final_fecha)}.*?(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"
        match = re.search(pattern_with_date, text, re.IGNORECASE | re.DOTALL)
        if match:
            return match.group(1).strip()

    # 2) Fallback: búsqueda sin fecha
    pattern_no_date = r"(POR\s+MEDIO\s+DE\s+LA\s+CUAL\s+.*?)(?:\.\s*|\n|EL\s+CONGRESO)"
    match = re.search(pattern_no_date, text, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()

    # Si ninguno de los dos patrones funcionó, retornamos None
    return None


## function to delete second estructure

In [1354]:
def remove_second_phrase_occurrence(text: str | None, phrase: str) -> str | None:
    """
    Elimina la segunda ocurrencia de `phrase` en `text`.
    Si `text` es None, se devuelve None directamente.
    """
    # 1) Si `text` es None o cadena vacía, no hacemos nada.
    if not text:
        return text

    # 2) Procedemos con la lógica de eliminación
    text_upper = text.upper()
    phrase_upper = phrase.upper()

    first_idx = text_upper.find(phrase_upper)
    if first_idx == -1:
        return text  # No aparece ni una vez

    # Buscar segunda ocurrencia a partir del final de la primera
    second_idx = text_upper.find(phrase_upper, first_idx + len(phrase))
    if second_idx == -1:
        return text  # Solo hay una ocurrencia

    # Eliminar la segunda ocurrencia (solo la frase en sí)
    before = text[:second_idx]
    after = text[second_idx + len(phrase):]
    return before + after


## function to extraction spacy 

In [1355]:
def looks_like_epigrafe(text_epigrafe: str) -> bool:
    """
    Verifica si la cadena `text_epigrafe` cumple criterios mínimos 
    para considerarse un epígrafe válido.
    """
    if not text_epigrafe:
        return False
    
    # 1) Longitud mínima y máxima (ajusta según tu caso)
    #    Por ejemplo, no menos de 30 caracteres ni más de 600.
    if len(text_epigrafe) < 30 or len(text_epigrafe) > 600:
        return False
    
    # 2) Que contenga la frase "POR MEDIO DE"
    #    (en mayúsculas para no depender del case)
    if "POR MEDIO DE" not in text_epigrafe.upper():
        return False
    
    return True


In [1356]:
def extract_metadata_layout_hybrid(text_lineal: str, nlp) -> dict:
    text_lineal = fix_ocr_issues(text_lineal)

    # 1) spaCy y regex de fecha
    spacy_result = spacy_extract_metadata(text_lineal, nlp)
    tipo_ley, ley_number = parse_ley_and_type(text_lineal)
    fecha_regex = parse_date_str(text_lineal)
    spacy_fecha = spacy_result["fecha"]
    spacy_fecha_normalizada = normalize_spacy_date(spacy_fecha)

    if spacy_fecha_normalizada:
        final_fecha = spacy_fecha_normalizada
    else:
        final_fecha = fecha_regex

    # 2) Epígrafe por fallback
    epigrafe_regex = extraer_epigrafe_fallback(text_lineal, final_fecha)

    # 3) Número de ley (fallback)
    spacy_numero_ley = spacy_result["numero_ley"]
    if not spacy_numero_ley or not looks_like_number_ley(spacy_numero_ley):
        spacy_numero_ley = ley_number

    # 4) Epígrafe (fallback spaCy vs regex)
    spacy_epigrafe = spacy_result["epigrafe"]
    if spacy_epigrafe and looks_like_epigrafe(spacy_epigrafe):
        final_epigrafe = spacy_epigrafe
    else:
        final_epigrafe = epigrafe_regex

    # 8) Remover segunda ocurrencia
    final_epigrafe = remove_second_phrase_occurrence(final_epigrafe, "POR MEDIO DE LA CUAL")


    combined = {
        "numero_ley": spacy_numero_ley,
        "fecha": final_fecha,
        "tipo_ley": tipo_ley,
        "epigrafe": final_epigrafe,
    }
    return combined


In [1357]:

def extract_metadata_from_pdf_all_pages(pdf_path: str, nlp=None) -> dict:
    """
    Procesa todas (o varias) páginas del PDF y combina los metadatos 
    en un único diccionario. 
    Si encuentra datos faltantes en la primera página, intenta completarlos 
    en las siguientes.
    """
    # Diccionario final, inicialmente vacío
    final_metadata = {
        "numero_ley": None,
        "fecha": None,
        "tipo_ley": None,
        "epigrafe": None
    }

    with pdfplumber.open(pdf_path) as pdf:
        for page_index, page in enumerate(pdf.pages):
            page_text = page.extract_text() or ""
            page_text = fix_ocr_issues(page_text)

            # Extraer metadatos de ESTA página
            partial_metadata = extract_metadata_layout_hybrid(page_text, nlp)

            # Combinar
            # Si 'final_metadata' no tiene un valor, y 'partial_metadata' sí, lo copiamos
            for key in final_metadata.keys():
                if final_metadata[key] is None and partial_metadata[key] is not None:
                    final_metadata[key] = partial_metadata[key]

            # (Opcional) Si ya tenemos todos los campos, rompemos el bucle
            if all(final_metadata[k] is not None for k in final_metadata):
                break

    return final_metadata


## format lecture a pdf

In [1358]:
def extract_metadata_from_pdf_first_page(pdf_path: str, nlp=None) -> dict:
    with pdfplumber.open(pdf_path) as pdf:
        first_page = pdf.pages[0]
        page_text = first_page.extract_text() or ""

    page_text = fix_ocr_issues(page_text)

    if nlp:
        metadata = extract_metadata_layout_hybrid(page_text, nlp)
    else:
        metadata = spacy_extract_metadata(page_text)

    return metadata


In [1359]:
# test_text = (
#     '2353 17ABR2024 LEY  No. "POR  MEDIO  DE  LA  CUAL  SE  RECONOCE  COMO  PATRIMONIO '
#     'CULTURAL  INMATERIAL  DE  LA NACION  EL  FESTIVAL  PROVINCIANO  DE  ACORDEONES,  CANCION '
#     'INÉDITA  Y  PIQUERIA,  DEL  MUNICIPIO  DE  PIVIJAY  -  MAGDALENA  Y  SE  DICTAN  OTRAS '
#     'DISPOSICIONES" El  Congreso  de  Colombia,'
# )
# doc = nlp_trained(test_text)
# for ent in doc.ents:
#     print(ent.text, ent.label_)

## use

In [1360]:
def leer_pdf_primera_pagina(ruta_pdf: str) -> str:
    with pdfplumber.open(ruta_pdf) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text() or ""
    return text

In [1361]:
def extraer_metadatos_gaceta_spacy(texto: str) -> dict:
    """
    Envuelve la llamada a tu función híbrida, 
    para que sea fácil de usar en el bucle de PDFs.
    """
    # Aquí usas la función híbrida con tu modelo entrenado
    metadatos = extract_metadata_layout_hybrid(texto, nlp_trained)
    return metadatos

In [1362]:
def procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida):
    """Procesa todos los archivos PDF en una carpeta y guarda los metadatos en archivos JSON separados."""
    for archivo in os.listdir(carpeta_pdf):
        if archivo.lower().endswith(".pdf"):
            ruta_pdf = os.path.join(carpeta_pdf, archivo)
            
            # 1) Lee el PDF (primera página o todas)
            texto_pdf = leer_pdf_primera_pagina(ruta_pdf)
            # texto_pdf = leer_pdf_todas_paginas(ruta_pdf)  # si quieres todas las páginas
            
            # 2) Extrae los metadatos con tu pipeline híbrida
            metadatos = extraer_metadatos_gaceta_spacy(texto_pdf)
            
            # 3) Guarda en JSON
            nombre_json = os.path.splitext(archivo)[0] + ".json"
            ruta_json = os.path.join(carpeta_salida, nombre_json)
            with open(ruta_json, "w", encoding="utf-8") as f:
                json.dump(metadatos, f, ensure_ascii=False, indent=4)
            
            print(f"Metadatos guardados en: {ruta_json}")

In [1363]:
# --- Ejemplo de Uso --- # 20190620 LEY 1959 # LEY-2199-2022 
carpeta_pdf = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/leyes"
carpeta_salida = r"c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024"
os.makedirs(carpeta_salida, exist_ok=True)

procesar_pdfs_en_carpeta(carpeta_pdf, carpeta_salida)

Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2199-2022.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2350-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2351-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2352-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2353-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2354-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2355-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2356-2024.json
Metadatos guardados en: c:/Users/Jorge/OneDrive/Documents/proyect/document/json_output_2024\LEY-2358-2024.json
M

In [1364]:
# import pdfplumber

# pdf_path = r"C:\Users\Jorge\OneDrive\Documents\proyect\document\leyes\LEY-2199-2022.pdf"

# all_text = ""
# with pdfplumber.open(pdf_path) as pdf:
#     for i, page in enumerate(pdf.pages):
#         page_text = page.extract_text() or ""
#         print(f"--- Página {i} ---")
#         print(repr(page_text))
#         print("---------------")
#         all_text += page_text + "\n"